In [1]:
# parse_md.py
from pathlib import Path
import re

def load_articles(md_path: str):
    text = Path(md_path).read_text(encoding="utf-8")
    # Разделяем по --- между статьями
    chunks = [c.strip() for c in re.split(r'\n-{3,}\n', text) if c.strip()]

    articles = []
    for i, ch in enumerate(chunks, 1):
        # Заголовок = первая строка, начинающаяся с '#'
        m = re.match(r'^\s*#\s*(.+)\s*$', ch.splitlines()[0])
        title = m.group(1).strip() if m else f"Article {i}"
        body = "\n".join(ch.splitlines()[1:]).strip()
        articles.append({
            "id": f"a{i:04d}",
            "title": title,
            "content": body
        })
    return articles

if __name__ == "__main__":
    arts = load_articles("data.md")
    print(f"Loaded {len(arts)} articles")
    print(arts[0]["title"])


Loaded 8 articles
How to use Single Sided Connectors?


In [2]:
# index_build.py
from whoosh import index
from whoosh.fields import Schema, TEXT, ID
# ИЛИ StemmingAnalyzer(), ИЛИ LanguageAnalyzer("en")
from whoosh.analysis import LanguageAnalyzer
from whoosh.writing import AsyncWriter
from pathlib import Path

def build_index(md_path: str, index_dir: str = "index"):
    analyzer = LanguageAnalyzer("en")  # <— вот тут фикс
    schema = Schema(
        id=ID(stored=True, unique=True),
        title=TEXT(stored=True, analyzer=analyzer),
        content=TEXT(stored=True, analyzer=analyzer),
    )

    idx_path = Path(index_dir)
    if not idx_path.exists():
        idx_path.mkdir(parents=True, exist_ok=True)
        ix = index.create_in(index_dir, schema)
    else:
        # если схема изменилась — лучше пересоздать индекс начисто
        try:
            ix = index.open_dir(index_dir)
        except:
            for p in idx_path.glob("*"):
                p.unlink()
            ix = index.create_in(index_dir, schema)

    arts = load_articles(md_path)
    writer = AsyncWriter(ix)
    for a in arts:
        writer.update_document(
            id=a["id"],
            title=a["title"],
            content=a["content"]
        )
    writer.commit()
    return ix

if __name__ == "__main__":
    build_index("data.md")
    print("Index built ✅")


Index built ✅


In [2]:
# search_run.py
from whoosh import index
from whoosh.qparser import MultifieldParser, OrGroup
from whoosh.scoring import BM25F
from whoosh.highlight import ContextFragmenter, WholeFragmenter, HtmlFormatter

SYN = {
    "connector": ["connector", "connectors", "fitting", "joint"],
    "screw": ["screw", "screws"],
    "traverse": ["traverse", "traverses", "cross beam", "cross beams"],
    "woodwop": ["woodwop", "mpr"],
    "back panel": ["back panel", "rebated", "rebating"],
    "pocket": ["pocket", "pockets", "milling"],
}

def expand_query(q: str):
    tokens = q.lower().split()
    expanded = []
    for t in tokens:
        expanded += SYN.get(t, [t])
    return " ".join(set(expanded))

def _safe_preview(text: str, limit: int = 600) -> str:
    """Мягкое превью: режем по границе предложения/слова, добавляем многоточие."""
    if len(text) <= limit:
        return text
    cut = text[:limit]
    # попробуем обрезать по последней точке/новой строке/пробелу
    for sep in [". ", "\n", " "]:
        pos = cut.rfind(sep)
        if pos > limit * 0.6:
            return cut[:pos].rstrip() + "…"
    return cut.rstrip() + "…"

def search(query: str, index_dir: str = "index", top_k: int = 5, full: bool = False):
    """
    full=False -> возвращаем snippet_html + preview
    full=True  -> возвращаем весь контент с подсветкой (осторожно, длинно!)
    """
    ix = index.open_dir(index_dir)
    with ix.searcher(weighting=BM25F(B=0.75, K1=1.5)) as s:
        fields = ["title", "content"]
        parser = MultifieldParser(fields, schema=ix.schema, group=OrGroup)
        q = parser.parse(expand_query(query))
        results = s.search(q)

        # 1) Форматтер: простой <mark> без class
        results.formatter = HtmlFormatter(tagname="mark", classname="", termclass="")

        if not full:
            results.fragmenter = ContextFragmenter(maxchars=1200, surround=180)
        else:
            results.fragmenter = WholeFragmenter()

        # 2a) Длинные контекстные фрагменты
        #    - увеличим окружение и общий максимум символов
        if not full:
            results.fragmenter = ContextFragmenter(maxchars=1200, surround=180)
        else:
            # 2b) Весь текст (осторожно с большим UI)
            results.fragmenter = WholeFragmenter()

        hits = []
        for r in results:
            # подсветанный фрагмент (или весь текст, если full=True)
            snippet_html = r.highlights("content", top=3, minscore=1)

            # если подсветка пустая (редко), делаем мягкое превью вручную
            if not snippet_html:
                snippet_html = _safe_preview(r["content"])

            item = {
                "id": r["id"],
                "title": r["title"],
                "score": float(r.score),
                "snippet_html": snippet_html,      # HTML с <mark>, без классов
                "preview": _safe_preview(r["content"]),  # plain text, удобно для fallback
            }

            # по желанию отдаём полный текст (для «Показать всё»)
            if full:
                item["content_full"] = r["content"]  # Stored=True в схеме позволяет это делать

            hits.append(item)
        return hits

if __name__ == "__main__":
    out = search("woodwop component", top_k=3)
    print(out[0]["snippet_html"])

To use a <mark class=" 0">woodWOP</mark> <mark class=" 1">component</mark> in your SmartWOP project navigate to the Machinings tab. Click on <mark class=" 0">woodWOP</mark> <mark class=" 2">Component</mark> in the Free Machinings section.

Now the properties window for this special machining type opens.

By default the file is named external.mpr this comes preinstalled with SmartWOP and is ready to be used and edited...To create a new <mark class=" 0">woodWOP</mark> <mark class=" 1">component</mark>, change the file name or select an existing <mark class=" 0">woodWOP</mark> <mark class=" 1">component</mark> using the icon with the three dots to browse your computer.

Clicking on the <mark class=" 0">woodWOP</mark> icon starts <mark class=" 0">woodWOP</mark>. You can only edit <mark class=" 3">components</mark> in <mark class=" 0">woodWOP</mark> if you have previously unchecked the вЂњEmbeddedвЂќ option.

In <mark class=" 0">woodWOP</mark> you can now edit the <mark class=" 1">component

In [9]:
import os
from typing import List, Dict, Optional
from openai import OpenAI
from openai import BadRequestError

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

SYSTEM_INSTRUCTIONS = (
    "You are a helpful assistant for a CAD app (SmartWOP). "
    "Answer ONLY using the provided context. If the answer is not in context, say you don't know. "
    "Prefer step-by-step, concise instructions. Keep tool/command names verbatim."
)

def build_context_blocks(hits: List[Dict]) -> str:
    blocks = []
    for i, h in enumerate(hits, 1):
        blocks.append(f"[{i}] {h['title']}\n{h['snippet_html']}")
    return "\n\n".join(blocks)

def _create_response_safe(*, model: str, instructions: str, input_payload, 
                          max_output_tokens: int = 400, temperature: Optional[float] = None):
    kwargs = dict(
        model=model,
        instructions=instructions,
        input=input_payload,
        max_output_tokens=max_output_tokens,
        store=False,
    )
    # Пробуем с temperature, если задана
    if temperature is not None:
        kwargs["temperature"] = temperature
        try:
            return client.responses.create(**kwargs)
        except BadRequestError as e:
            # Если модель не поддерживает temperature — повторим без него
            if "Unsupported parameter" in str(e) and "temperature" in str(e):
                kwargs.pop("temperature", None)
                return client.responses.create(**kwargs)
            raise
    else:
        # Сразу без temperature
        return client.responses.create(**kwargs)

def ask_llm_with_rag(user_query: str, top_k: int = 5, 
                     model: str = "gpt-4o-mini", temperature: Optional[float] = 0.2) -> str:
    hits = search(user_query)
    context = build_context_blocks(hits)

    input_payload = [
        {
            "role": "user",
            "content": [
                {"type": "input_text",
                 "text":
                    f"User question (keep language of the user): {user_query}\n\n"
                    f"=== CONTEXT START ===\n{context}\n=== CONTEXT END ===\n\n"
                    f"Rules:\n"
                    f"- Answer in the same language as the user question.\n"
                    f"- Cite sources as [1], [2] referring to the blocks above (no external links).\n"
                    f"- If not enough info in context, try to help."
                 }
            ],
        }
    ]

    resp = _create_response_safe(
        model=model,
        instructions=SYSTEM_INSTRUCTIONS,
        input_payload=input_payload,
        max_output_tokens=300,
        temperature=temperature,   # будет удалён автоматически, если не поддерживается
    )
    return resp.output_text

# Пример вызова:
if __name__ == "__main__":
    # Пример
    q = "How Installing rebated back panels.?"
    try:
        print(ask_llm_with_rag(q))
    except NotImplementedError:
        print("Подключи свою функцию search() из BM25 и запусти снова.")


To install rebated back panels, follow these steps:

1. **Select the Back Panel Template**: This template allows you to modify the basic values of the back panel.

2. **Adjust Thickness and Setback**: Set the thickness of the back panel and the setback. For example, a setback of 20 mm with an 8 mm back panel results in an actual setback of 12 mm.

3. **Determine Overlap**: Decide how far the back panel will insert into your carcass panels.

4. **Set Rebating Options**: Use the Back Panel Rebating feature, which is particularly useful for thicker back panels. Adjust the front and back distances as needed.

5. **Configure Air Gap Width**: Set the air gap width to allow for easier installation around the rebate.

6. **Machine on CNC**: If you want SmartWOP to write a program for the back panel, ensure "Machine on CNC" is selected. If selected, choose a Tool Number from the dropdown menu and set a Custom Feedrate if necessary.

By following these steps, you can successfully install rebated